In [ ]:
! sudo apt-get install g++ openjdk-7-jdk
! sudo apt-get install python3-dev; pip3 install konlpy

! git clone https://github.com/lovit/customized_konlpy.git
! pip install customized_konlpy

!pip install pyldavis

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package openjdk-7-jdk is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'openjdk-7-jdk' has no installation candidate
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.6.7-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.
fatal: destination path 'customized_konlpy' already exists and is not an empty directory.


In [ ]:
# 데이터 모듈
import os
import pickle
import pandas as pd
from glob import glob
import re
import numpy as np
import pandas as pd
from tqdm import tqdm

import Tokenize_with_TfIdf_v3 as twt

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim

from pprint import pprint

import warnings
warnings.filterwarnings(action='ignore') 

import matplotlib.pyplot as plt

# 그래프 그리기 위한 plotly
import plotly.express as px

import plotly.io as pio
pio.renderers.default = "notebook_connected"

import plotly.offline as pyo
#import plotly.graph_objs as go
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

In [ ]:
# https://dacon.io/competitions/official/235618/codeshare/1513?page=2&dtype=vote&ptype=pub
import matplotlib.font_manager as fm
import matplotlib as mpl
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum

font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=font_path, size=5)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()
mpl.rcParams['axes.unicode_minus'] = False

# Colab 나눔고딕 설정

plt.rc('font', family='NanumBarunGothic')

In [ ]:
# colab plotly display 위한 코드
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
            },
          });
        </script>
        '''))

# 코로나 바이러스 감염증 국내 발생 현황

In [ ]:
covid_data = pd.read_excel('./data/코로나바이러스감염증-국내발생현황(data_only).xlsx')

In [ ]:
covid_data

,날짜,격리해제(퇴원),확진자 수,사망,검사중,결과 음성,검사자(계),확진자(계),총(계),일자별 확진자
0,2020-01-20,0,1,0,0,0,0,1,1,1
1,2020-01-21,0,1,0,0,0,0,1,1,0
2,2020-01-22,0,1,0,0,0,0,1,1,0
3,2020-01-23,0,1,0,0,0,0,1,1,0
4,2020-01-24,0,2,0,0,0,0,2,2,1
...,...,...,...,...,...,...,...,...,...,...
270,2020-10-16,23180,1414,441,17355,2417036,2434391,25035,2459426,47
271,2020-10-17,23258,1407,443,17318,2426101,2443419,25108,2468527,73
272,2020-10-18,23312,1443,444,17781,2431549,2449330,25199,2474529,91
273,2020-10-19,23368,1463,444,16763,2437188,2453951,25275,2479226,76


In [ ]:

init_notebook_mode(connected=True)

configure_plotly_browser_state()

# 코로나 감염자 수 변화 그래프
coronagra = go.Bar(
    x = covid_data['날짜'].astype('str'),
    y = covid_data['일자별 확진자'],
    marker=dict(color="light blue"),
    name='확진자 수',
    )

# 31번 확진자 확진날짜 표시
corona_31 =  go.Scatter(x=['2020-02-18', '2020-02-18'],
                            y=[0,850],
                            mode="lines",
                            legendgroup="a",
                            showlegend=True,
                            marker=dict(size=12,
                                       line=dict(width=0.8),
                                       color="red"
                                       ),
                            name="31번 확진자 확진일",
                            )

# 구로 콜센터 최초 확진날짜 표시
corona_call =  go.Scatter(x=['2020-03-08', '2020-03-08'],
                            y=[0,850],
                            mode="lines",
                            legendgroup="a",
                            showlegend=True,
                            marker=dict(size=12,
                                       line=dict(width=0.8),
                                       color="gold"
                                       ),
                            name="구로 콜센터 최초 확진자 확진일",
                            )


# 이태원 집단감역 최초 확진자 확진날짜 표시
corona_itaewon =  go.Scatter(x = ['2020-05-06', '2020-05-06'],
                            y=[0,850],
                            mode="lines",
                            legendgroup="a",
                            showlegend=True,
                            marker=dict(size=12,
                                       line=dict(width=0.8),
                                       color="green"
                                       ),
                            name="이태원 집단발병 시작(용인-66)",
                            )
data_comp = [coronagra, corona_31, corona_call, corona_itaewon]

# 수도권 교회 집단감염 사건 발생일
corona_seoul =  go.Scatter(x = ['2020-08-15', '2020-08-15'],
                            y=[0,850],
                            mode="lines",
                            legendgroup="a",
                            showlegend=True,
                            marker=dict(size=12,
                                       line=dict(width=0.8),
                                       color="Orange"
                                       ),
                            name="수도권 교회 집단감염 사건",
                            )
data_comp = [coronagra, corona_31, corona_call, corona_itaewon, corona_seoul]

# x축, y축, 타이틀 설정
layout_comp = go.Layout(
    title='코로나 감염자 수 변화(2020/01/20 - 2020/10/20)',
    hovermode='closest',
    xaxis=dict(
        title='확진날짜',
        ticklen=5,
        zeroline=False,
        gridwidth=2,
        range=['2020-01-20', '2020-10-20'],
    ),
    yaxis=dict(
        title='확진자수',
        ticklen=5,
        gridwidth=2,
        range=[0, 850],
    ),
)
fig_comp = go.Figure(data=data_comp, layout=layout_comp)
iplot(fig_comp)

In [ ]:
configure_plotly_browser_state()

fig = go.Figure()
fig.add_trace(go.Scatter(x=covid_data['날짜'].astype('str'), y=covid_data['확진자(계)'],name='확진자(계)'))
fig.add_trace(go.Scatter(x=covid_data['날짜'].astype('str'), y=covid_data['검사자(계)'],name='검사자(계)'))
fig.add_trace(go.Scatter(x=covid_data['날짜'].astype('str'), y=covid_data['사망'],name='사망'))

# 보건용 마스크 생산 수급 현황

In [ ]:
mask_data = pd.read_excel('./data/보건용 마스크 생산 수급 현황.xlsx')

In [ ]:
configure_plotly_browser_state()

mask_data[['구분(단위:만개)','총계','약국','우체국','의료기관']]

fig = go.Figure()
fig.add_trace(go.Scatter(x=mask_data['구분(단위:만개)'], y=mask_data['총계'], fill='tozeroy',name='총계'))
fig.add_trace(go.Scatter(x=mask_data['구분(단위:만개)'], y=mask_data['약국'], fill='tozeroy',name='약국'))
fig.add_trace(go.Scatter(x=mask_data['구분(단위:만개)'], y=mask_data['우체국'], fill='tozeroy',name='우체국'))
fig.add_trace(go.Scatter(x=mask_data['구분(단위:만개)'], y=mask_data['의료기관'], fill='tozeroy',name='의료기관'))

fig.update_layout(title='공적 마스크',
                   xaxis_title='일자',
                   yaxis_title='단위:만개')

fig.show()

# 분석 결과 연계 키워드 대상 대시보드 기능 내장

In [ ]:
# 데이터 로드, null 처리, 미사용 열 제거
data = pd.read_csv('./data/KCI.csv')
data = data.dropna(subset = ['original_abstract'])
data = data.drop(['Unnamed: 0','foreign_title','english_title','english_abstract','url'],axis=1)

In [ ]:
data.pub_date.unique()

array([2020.05, 2020.08, 2020.03, 2020.04, 2020.06, 2020.07])

In [ ]:
def tokenized_list(data):
  data_1 = data.original_abstract

  # 한글 정규 표현식
  hanspell = re.compile(r'[가-힣]{1}')

  # abstrack 의 길이가 10 이상, 첫 10글자에 한글이 존재(영문 요약이 아니면) 추가
  data_2 = []
  for doc_ in data_1:
    if (len(doc_) > 10 ) and (re.search(hanspell,doc_[:10])):
      data_2.append(doc_)

  # 형태소 분석에 몇몇 새로운 단어 추가
  word_list =['COVID-19','코로나19','코로나바이러스감염증-19','중증급성호흡기증후군',
              '고강도','사회적','확진자','거리두기','소모임','전파력','세계적','차선책','주도적','특별검역절차','특별입국절차','산업체','비대면','관광산업','제4차산업혁명','인공지능',
              'SARS-CoV-2','SARS','MERS']
  for word in word_list:
      twt.AddWord(word,'Noun')
  
  text_values = twt.Tokenize_list(data_2)

  # 불용어 처리 과정
  Tokenize = text_values
  display_number = -1
  tokenize_ = Tokenize.copy()
  with open('./stopword.pkl', 'rb') as f:
      stopn_list = pickle.load(f)
  remove_words = []
  for num_i, doc in enumerate(tokenize_):        
      for num_j ,target_word in enumerate(doc):
          if target_word in stopn_list or len(target_word) == 1:
              remove_words.append((target_word ,num_i ,num_j ))
  remove_words.reverse()
  for r_word in remove_words:
      del tokenize_[r_word[1]][r_word[2]]
  
  return tokenize_

In [ ]:
data_3 = tokenized_list(data)

## LDA 적용

In [ ]:
id2word = corpora.Dictionary(data_3)
texts = data_3
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                            id2word = id2word,
                                            num_topics = 20,
                                            random_state = 100,
                                            update_every =1,
                                            chunksize = 100,
                                            passes = 10,
                                            alpha= 'auto',
                                            per_word_topics = True
                                            )

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pyLDAvis.display(vis)

# 소셜 SNS A.I 가공 기반 사회 지수 생성

In [ ]:
df = pd.read_csv('./data/1차.csv')
df = df.drop('Unnamed: 0',axis=1)
df = df.drop_duplicates()
df.head()

,doc_id,depth1_nm,depth2_nm,depth3_nm,doc_title,index_y,index_m,index_d,topics
0,ac5a89693ca340b26f2840720ce759be,미디어,기타,네이버뉴스,트럼프 코로나TF 유지 해체 언급 하루만에 번복,2020,5,7,"['연합뉴스tv', '태스크포스', '트럼프', '코로나', '뉴스도널드', '해체..."
1,e53f3a951f766ac9dda2fae0319b3b47,미디어,기타,네이버뉴스,클럽노래방 이제 QR코드 찍고 들어간다,2020,5,31,"['고위험시설', '전자출입명부', '다음달', '중앙재난안전대책본부', '경향신문..."
2,563e2b8f174dc359f49106670d9398c8,미디어,기타,네이버뉴스,날씨 이야기 지구의 주인,2020,4,18,"['미생물', '동아일보', '생명', '온실가스', '미세먼지', '기념', '기..."
3,48402fbb24d0564f6ecaca3ec4ca9ca0,미디어,기타,네이버뉴스,"본토 코로나19 이틀째 0명, 역유입은 39명",2020,3,20,"['코로나', '확진자', '역유입', '사망자', '본토', '신규', '역유입되..."
7,35ac30f25d68f1b1abd709c9cfbcf303,미디어,기타,네이버뉴스,용인시민 1명 서울서 코로나19 양성 판정,2020,5,24,"['용인', '코로나', '성동구보건소', '서울', '처인구', '포곡읍', '실..."


In [ ]:
# 군산대 감성분석 사전 활용
# https://github.com/park1200656/KnuSentiLex
with open('./data/SentiWord_info.json', encoding='utf-8-sig', mode='r') as f:
	data = json.load(f)
po_ne_dict = {}
for each_ in data:
  po_ne_dict[each_['word']] = each_['polarity']

In [ ]:
def positive_negative(word_list):
  po_result = 0
  ne_result = 0
  for word in word_list:
    try:
      words_pone = int(po_ne_dict[word])
    except:
      continue
    if words_pone > 0:
      po_result += words_pone
    elif words_pone < 0:
      ne_result += words_pone
  return po_result,ne_result

In [ ]:
tqdm.pandas()
df['po_ne'] = df['topics'].progress_apply(lambda x : positive_negative(x))

100%|██████████| 643600/643600 [00:16<00:00, 39841.71it/s]


In [ ]:
df.head()

,doc_id,depth1_nm,depth2_nm,depth3_nm,doc_title,index_y,index_m,index_d,topics,po_ne
0,ac5a89693ca340b26f2840720ce759be,미디어,기타,네이버뉴스,트럼프 코로나TF 유지 해체 언급 하루만에 번복,2020,5,7,"['연합뉴스tv', '태스크포스', '트럼프', '코로나', '뉴스도널드', '해체...","(1, -2)"
1,e53f3a951f766ac9dda2fae0319b3b47,미디어,기타,네이버뉴스,클럽노래방 이제 QR코드 찍고 들어간다,2020,5,31,"['고위험시설', '전자출입명부', '다음달', '중앙재난안전대책본부', '경향신문...","(0, 0)"
2,563e2b8f174dc359f49106670d9398c8,미디어,기타,네이버뉴스,날씨 이야기 지구의 주인,2020,4,18,"['미생물', '동아일보', '생명', '온실가스', '미세먼지', '기념', '기...","(0, -2)"
3,48402fbb24d0564f6ecaca3ec4ca9ca0,미디어,기타,네이버뉴스,"본토 코로나19 이틀째 0명, 역유입은 39명",2020,3,20,"['코로나', '확진자', '역유입', '사망자', '본토', '신규', '역유입되...","(0, 0)"
7,35ac30f25d68f1b1abd709c9cfbcf303,미디어,기타,네이버뉴스,용인시민 1명 서울서 코로나19 양성 판정,2020,5,24,"['용인', '코로나', '성동구보건소', '서울', '처인구', '포곡읍', '실...","(0, -4)"


In [ ]:
from plotly.subplots import make_subplots

configure_plotly_browser_state()

fig = make_subplots(
                    rows=2, 
                    cols=6,
                   specs = [ [{"type":"domain"},{"type":"domain"},{"type":"domain"},{"type":"domain"},{"type":"domain"},{"type":"domain"}], 
                             [{"colspan":6},None,None,None,None,None]],
    subplot_titles = ("1월","2월","3월","4월","5월","6월","코로나에 대한 공포지수 변화 추이(2020.01.2020.06.)")
   )

for i in range(1,7):
    tmp = df[df.index_m == i]
    pos = sum([po[0] for po in tmp.po_ne])
    neg = np.abs(sum([po[1] for po in tmp.po_ne]))

    fig.add_trace(
            go.Pie(
                labels=["Positive","Negative"], 
                values=[pos, neg], 
                hole=.3
            ),
        row = 1, 
        col = i
    )

y = []
for i in range(1,7):
  y.append(sum([num[0] + num[1]  for num in df[df.index_m == i].po_ne]))

fig.add_trace(
        go.Scatter(
            x = ["1월","2월","3월","4월","5월","6월"],
            y = y,
            mode = "lines+markers",
            name = "Score",
            fill      = 'tozeroy',
            fillcolor = "rgba(51, 158, 188, 0.7)",
            marker_color = "rgba(51, 158, 188, 0.7)",
            line = dict(smoothing = 1.3, shape="spline")
        ),
                row = 2,
                col = 1
)

fig.update_layout(showlegend=False, shapes = [] )
fig.show()